In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers.convolutional import Conv2D 
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.datasets import mnist
from keras.utils import np_utils
from keras.optimizers import SGD, RMSprop, Adam
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
from keras.callbacks import TensorBoard,ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from sklearn.cross_validation import train_test_split
from keras.layers.wrappers import Bidirectional, TimeDistributed
from keras.layers import ConvLSTM2D,Input
from keras.models import Model
from keras.engine.topology import Layer,InputSpec
K.set_image_dim_ordering("th")


/home/rjpg/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/rjpg/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
sess = tf.Session()
K.set_session(sess)


dfin = pd.read_csv('../NNNormalizeDataIn.csv',header=None)  #inputs 128 lines per sample
dfout = pd.read_csv('../NNNormalizeDataOutClassses.csv',header=None)  #output classes 

#dfin=dfin.drop(dfin.columns[[2,3,4]], axis=1) #take some indicators out

NVARS=9

total_inputs,total_output = dfin.as_matrix().astype(np.float32),dfout.as_matrix().astype(np.int32)

print(total_inputs.shape)
print(total_output.shape)

print(total_inputs[0:128])



total_inputs = np.reshape(total_inputs, (-1,128,9))
print("######################################################")
print(total_inputs[0])
total_inputs = np.reshape(total_inputs, (-1,8,16,9))
print("######################################################")
print(total_inputs[0,7,15])


(265600, 9)
(2075, 1)
[[ 1.7088607e-01 -1.6782573e-01 -2.4510670e-01 ...  1.9872051e-01
   3.2241258e-01  5.9588689e-01]
 [ 1.0000000e+00  1.6732299e-01  1.1027586e-01 ...  1.9872051e-01
   4.5077366e-01  5.0246048e-01]
 [-1.6222519e-01  3.3489737e-01  3.8603857e-01 ...  1.9872051e-01
   3.7737539e-01  3.8321662e-01]
 ...
 [ 4.3304465e-03 -2.5136155e-04 -4.0739995e-01 ...  9.8760493e-02
   1.7321697e-01  3.5674417e-01]
 [ 4.3304465e-03 -2.5136155e-04 -4.6310234e-01 ...  9.8760493e-02
   2.4600889e-01  3.4299701e-01]
 [-3.2878080e-01 -2.5136155e-04 -4.4819331e-01 ...  9.8760493e-02
   9.8892011e-02  3.4476128e-01]]
######################################################
[[ 1.7088607e-01 -1.6782573e-01 -2.4510670e-01 ...  1.9872051e-01
   3.2241258e-01  5.9588689e-01]
 [ 1.0000000e+00  1.6732299e-01  1.1027586e-01 ...  1.9872051e-01
   4.5077366e-01  5.0246048e-01]
 [-1.6222519e-01  3.3489737e-01  3.8603857e-01 ...  1.9872051e-01
   3.7737539e-01  3.8321662e-01]
 ...
 [ 4.3304465e-03 -2.5

/home/rjpg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [3]:
class CylindricalPadLSTMCNN2D(Layer):

    def __init__(self, m=0,n=1, **kwargs):
        super(CylindricalPadLSTMCNN2D, self).__init__(**kwargs)
        self.n = n
        self.m = m
        assert n > 0, 'n must be positive'
        
    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        super(CylindricalPadLSTMCNN2D, self).build(input_shape)  
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0],
                    input_shape[1],
                    input_shape[2],
                    input_shape[3] + 2*self.m,
                    input_shape[4] + 2*self.n)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0],
                    input_shape[1],
                    input_shape[2],
                    input_shape[3] + 2*self.m,
                    input_shape[4] + 2*self.n)
    
    def call(self,testElement, mask=None):
        firstColumns=testElement[:,:,:,:,0:self.n]
        lastColumns=testElement[:,:,:,:,testElement.shape[4]-self.n:testElement.shape[4]]
        result=tf.concat([testElement,firstColumns], axis=4)
        result=tf.concat([lastColumns,result], axis=4)
        if self.m != 0 :
            #x = tf.placeholder(result.dtype, shape=[result.shape[0],result.shape[1],self.m,result.shape[3]])
            #y = tf.zeros_like(x,dtype=result.dtype)
            
            firstRows=result[:,:,:,0:self.m,:]
            
            y = tf.fill(tf.shape(firstRows), 0.)
            
            #y=tf.fill([result.shape[0],result.shape[1],self.m,result.shape[3]],0.)
            
            #y = tf.constant(0., shape=[result.shape[0],result.shape[1],self.m,result.shape[3]],dtype=result.dtype)
            result=tf.concat([y,result], axis=3)
            result=tf.concat([result,y], axis=3)
        
        return result

    def get_config(self):
        config = {'n': self.n,
        'm': self.m}
        base_config = super(CylindricalPadLSTMCNN2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

print("CylindricalPadLSTMCNN2D class layer defined")

CylindricalPadLSTMCNN2D class layer defined


In [5]:
def non_shuffling_train_test_split(X, y, test_size=0.2):
    i = int((1 - test_size) * X.shape[0]) + 1
    X_train, X_test = np.split(X, [i])
    y_train, y_test = np.split(y, [i])
    return X_train, X_test, y_train, y_test



train_inputs, test_inputs, train_output, test_output = non_shuffling_train_test_split(total_inputs, total_output, test_size=0.2)

print(train_inputs.shape)
print(train_output.shape)



import tensorflow as tf
from keras import backend as K
from keras.optimizers import Adam
from keras.utils import np_utils

NB_EPOCH = 500
# network and training
BATCH_SIZE = 64
VERBOSE = 1
OPTIMIZER = Adam()
VALIDATION_SPLIT=0.2

TIMESTEPS = 8
IMG_ROWS, IMG_COLS = 16, NVARS # input image dimensions
NB_CLASSES = 5  # number of outputs = number of classes
INPUT_SHAPE = (1, IMG_ROWS, IMG_COLS)

X_train = train_inputs[:,:, np.newaxis, :, :]
y_train = np_utils.to_categorical(train_output, NB_CLASSES)
X_test = test_inputs[:,:, np.newaxis, :, :]
y_test = np_utils.to_categorical(test_output, NB_CLASSES)

print(X_train.shape)


class convLSTMNet:
    @staticmethod
    def build(timeSteps,rows, cols,classes):
        #CONV=>POOL
        inputNet = Input(shape=(timeSteps,1,rows, cols)) #batch_shape=(20, 7, 5) 
        roll=CylindricalPadLSTMCNN2D(0,4)(inputNet)
        convlstm2d=Bidirectional(ConvLSTM2D(20, kernel_size=(4,9), padding="valid", dropout=0.3, recurrent_dropout=0.2,return_sequences=True))(roll)
        roll=CylindricalPadLSTMCNN2D(0,4)(convlstm2d)
        convlstm2d=Bidirectional(ConvLSTM2D(10, kernel_size=(6,9), padding="valid", dropout=0.3, recurrent_dropout=0.2))(roll)
        
        ##lstm=Bidirectional(LSTM(100,recurrent_dropout=0.4,dropout=0.4,return_sequences=True),merge_mode='concat')(inputNet) #worse using stateful=True
        
        #lstm=SeqSelfAttention(attention_activation='sigmoid')(lstm)
        
        ##lstm=Bidirectional(LSTM(50,recurrent_dropout=0.4,dropout=0.4,return_sequences=False),merge_mode='concat')(lstm) #worse using stateful=True 
        
        #classificationLayer=AttentionDecoder(150, classes)
        #lstm=LSTM(100,recurrent_dropout=0.4,dropout=0.4,return_sequences=True)(inputNet) #worse using stateful=True
        #lstm=LSTM(50,recurrent_dropout=0.4,dropout=0.4)(lstm) #worse using stateful=True 
        
        
        #denselayers=Dense(400)(lstm)
        #denselayers=Activation("relu")(denselayers)
        #denselayers=Dropout(0.5)(denselayers)
        #denselayers=Dense(150)(denselayers)
        #denselayers=Activation("relu")(denselayers)
        #denselayers=Dropout(0.8)(denselayers)
        # a softmax classifier
        lstm=Flatten()(convlstm2d)
        dense=Dense(100,activation="relu")(lstm)
        dense=Dropout(0.5)(dense)
        dense=Dense(50,activation="relu")(dense)
        dense=Dropout(0.5)(dense)
        classificationLayer=Dense(classes,activation='softmax')(dense)
        
        #classificationLayer=Activation("softmax")(classificationLayer)
        
        model=Model(inputNet,classificationLayer)
        return model
    
modellstm = convLSTMNet.build(timeSteps=8,rows=16, cols=NVARS,classes=5)
modellstm.summary()
modellstm.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
    metrics=["accuracy"])


# Prepare saver.
#builder = tf.saved_model.builder.SavedModelBuilder("./model_keras")

# Initialize all variables
sess.run(tf.global_variables_initializer())


(1661, 8, 16, 9)
(1661, 1)
(1661, 8, 1, 16, 9)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 8, 1, 16, 9)       0         
_________________________________________________________________
cylindrical_pad_lstmcn_n2d_3 (None, 8, 1, 16, 17)      0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 8, 20, 13, 18)     121120    
_________________________________________________________________
cylindrical_pad_lstmcn_n2d_4 (None, 8, 20, 13, 26)     0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 10, 8, 36)         129680    
_________________________________________________________________
flatten_2 (Flatten)          (None, 2880)              0         
_________________________________________________________________
dense_4 (Dense)              

In [6]:
tbCallBack = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
esCallBack = EarlyStopping(monitor='val_acc', min_delta=2, patience=50, verbose=0, mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.2,patience=5, min_lr=0.001)
best_checkpoint = ModelCheckpoint('lstmCÑNROLL_new_data.h5', monitor='val_acc', save_best_only=True, verbose=1)



history = modellstm.fit(X_train, y_train, 
        batch_size=BATCH_SIZE, epochs=NB_EPOCH, 
        verbose=2, # 0 for no logging to stdout, 1 for progress bar logging, 2 for one log line per epoch.
        #validation_split=VALIDATION_SPLIT,
        validation_data=(X_test,y_test),
        callbacks=[tbCallBack,best_checkpoint])#,reduce_lr,esCallBack])


Train on 1661 samples, validate on 414 samples
Epoch 1/500
 - 12s - loss: 1.6133 - acc: 0.2089 - val_loss: 1.6086 - val_acc: 0.2150

Epoch 00001: val_acc improved from -inf to 0.21498, saving model to lstmCÑNROLL_new_data.h5
Epoch 2/500
 - 7s - loss: 1.6131 - acc: 0.2053 - val_loss: 1.6103 - val_acc: 0.2126

Epoch 00002: val_acc did not improve from 0.21498
Epoch 3/500
 - 7s - loss: 1.6077 - acc: 0.2264 - val_loss: 1.6099 - val_acc: 0.1981

Epoch 00003: val_acc did not improve from 0.21498
Epoch 4/500
 - 7s - loss: 1.6082 - acc: 0.2312 - val_loss: 1.6114 - val_acc: 0.2319

Epoch 00004: val_acc improved from 0.21498 to 0.23188, saving model to lstmCÑNROLL_new_data.h5
Epoch 5/500
 - 7s - loss: 1.6103 - acc: 0.2059 - val_loss: 1.6085 - val_acc: 0.2174

Epoch 00005: val_acc did not improve from 0.23188
Epoch 6/500
 - 7s - loss: 1.6022 - acc: 0.2384 - val_loss: 1.6108 - val_acc: 0.1957

Epoch 00006: val_acc did not improve from 0.23188
Epoch 7/500
 - 7s - loss: 1.6023 - acc: 0.2300 - val_lo

 - 7s - loss: 0.9253 - acc: 0.6436 - val_loss: 2.0763 - val_acc: 0.2560

Epoch 00059: val_acc did not improve from 0.26812
Epoch 60/500
 - 7s - loss: 0.9134 - acc: 0.6550 - val_loss: 2.0987 - val_acc: 0.2440

Epoch 00060: val_acc did not improve from 0.26812
Epoch 61/500
 - 7s - loss: 0.8768 - acc: 0.6791 - val_loss: 2.1208 - val_acc: 0.2536

Epoch 00061: val_acc did not improve from 0.26812
Epoch 62/500
 - 7s - loss: 0.8492 - acc: 0.6875 - val_loss: 2.3499 - val_acc: 0.2367

Epoch 00062: val_acc did not improve from 0.26812
Epoch 63/500
 - 7s - loss: 0.8458 - acc: 0.6785 - val_loss: 2.1496 - val_acc: 0.2464

Epoch 00063: val_acc did not improve from 0.26812
Epoch 64/500
 - 7s - loss: 0.7946 - acc: 0.6893 - val_loss: 2.5086 - val_acc: 0.2271

Epoch 00064: val_acc did not improve from 0.26812
Epoch 65/500
 - 7s - loss: 0.7764 - acc: 0.6954 - val_loss: 2.3055 - val_acc: 0.2319

Epoch 00065: val_acc did not improve from 0.26812
Epoch 66/500
 - 7s - loss: 0.8115 - acc: 0.6978 - val_loss: 2

KeyboardInterrupt: 

In [7]:
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix

modelLoaded = load_model('lstmCÑNROLL_new_data.h5', custom_objects={'CylindricalPadLSTMCNN2D':CylindricalPadLSTMCNN2D})

Y_pred_L = modelLoaded.predict(X_test)
y_pred_L = np.argmax(Y_pred_L, axis=1)

print('Confusion Matrix')
cm_CNN=confusion_matrix(np.argmax(y_test,axis=1), np.argmax(modelLoaded.predict(X_test),axis=1))
print(cm_CNN)



print(Y_pred_L.shape)
print(y_test.shape)

Confusion Matrix
[[15 22  7 20  5]
 [17 28  5 20 16]
 [13 18  6 25 21]
 [ 7 16 11 31 26]
 [ 3 13  7 31 31]]
(414, 5)
(414, 5)


In [8]:
print(cm_CNN[0,0]+cm_CNN[0,1]+cm_CNN[1,0]+cm_CNN[1,1]+cm_CNN[3,3]+cm_CNN[3,4]+cm_CNN[4,3]+cm_CNN[4,4])
print(cm_CNN[3,0]+cm_CNN[3,1]+cm_CNN[4,0]+cm_CNN[4,1]+cm_CNN[0,3]+cm_CNN[0,4]+cm_CNN[1,3]+cm_CNN[1,4])

profit_CNN=(cm_CNN[0,0]+cm_CNN[0,1]+cm_CNN[1,0]+cm_CNN[1,1]+cm_CNN[3,3]+cm_CNN[3,4]+cm_CNN[4,3]+cm_CNN[4,4])-(cm_CNN[3,0]+cm_CNN[3,1]+cm_CNN[4,0]+cm_CNN[4,1]+cm_CNN[0,3]+cm_CNN[0,4]+cm_CNN[1,3]+cm_CNN[1,4])

print (profit_CNN)

201
100
101
